## Escolher uma das colunas dos dados utilizados no projeto 1 para predição (classificação ou regressão)

In [1]:
# !pip install mlflow
# !pip install --upgrade setuptools
# !pip install scikit-optimize

In [2]:
import pandas as pd
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import cross_val_score

In [3]:
path_raw = os.getenv('PATH_RAW')
# df_raw = pd.read_csv('scraped_data.csv', sep=',')
df_raw = pd.read_csv('../scraped_data.csv', sep=',', nrows=1000)

In [4]:
df_raw.head(1)

,find_anuncio,find_condominio,find_descri,find_endereco,find_iptu,find_loc,find_m_quadrado,find_price,find_vendedor,is_academia,...,is_mobiliado,is_piscina,is_portaria,is_profissional,is_salao_festa,is_varanda,n_banheiros,n_fotos,n_garagens,n_quartos
0,Apartamento para venda com 105 metros quadrado...,R$ 1.100 / mês,Código do anúncio: C6BAD9\n\nO apartamento est...,"Aflitos, Recife, PE, 52050340",R$ 180,Rua da Angustura,105m²,380000,Antônio Luciano Santana da Silva,False,...,False,False,False,True,False,False,3,20,0,3


In [5]:
df_0 = df_raw.copy()

In [6]:
df_0 = df_0.drop(columns=['find_anuncio','find_descri']) 

df_0 = df_0[df_0['n_quartos'] != '5 ou mais'] # por recomendação do professor
df_0 = df_0[df_0['n_banheiros'] != '5 ou mais']
df_0 = df_0[df_0['n_garagens'] != '5 ou mais']

df_0[['Bairro','CEP']] = df_0['find_endereco'].str.split(', ', expand=True)[[0,3]]
df_0 = df_0.drop(columns=['find_endereco'])

In [7]:
# deixando como num a coluna find_condominio
df_0['find_condominio'] = df_0['find_condominio'].astype('str').str.replace('R$', '').str.replace(' / mês', '').str.replace('.', '')

# deixando como num a coluna find_iptu
df_0['find_iptu'] = df_0['find_iptu'].astype('str').str.replace('R$ ', '').str.replace('.', '')

# deixando como num a coluna find_m_quadrado
df_0['find_m_quadrado'] = df_0['find_m_quadrado'].astype('str').str.replace('m²', '').str.replace('.', '')

# cast direto colunas(floats devido a conseguir dar cast em NaN)
df_0['find_condominio'] = df_0['find_condominio'].astype('float')
df_0['Bairro'] = df_0['Bairro'].astype('category')
df_0['CEP'] = df_0['CEP'].astype('category')
df_0['find_iptu'] = df_0['find_iptu'].astype('float')
df_0['find_loc'] = df_0['find_loc'].astype('category')
df_0['find_m_quadrado'] = df_0['find_m_quadrado'].astype('float')
df_0['find_price'] = df_0['find_price'].astype('float')
df_0['find_vendedor'] = df_0['find_vendedor'].astype('category')
df_0['is_academia'] = df_0['is_academia'].astype('category')
df_0['is_apertamento'] = df_0['is_apertamento'].astype('category')
df_0['is_ar_condicionado'] = df_0['is_ar_condicionado'].astype('category')
df_0['is_churrasqueira'] = df_0['is_churrasqueira'].astype('category')
df_0['is_elevador'] = df_0['is_elevador'].astype('category')
df_0['is_mobiliado'] = df_0['is_mobiliado'].astype('category')
df_0['is_piscina'] = df_0['is_piscina'].astype('category')
df_0['is_portaria'] = df_0['is_portaria'].astype('category')
df_0['is_profissional'] = df_0['is_profissional'].astype('category')
df_0['is_salao_festa'] = df_0['is_salao_festa'].astype('category')
df_0['is_varanda'] = df_0['is_varanda'].astype('category')
df_0['n_banheiros'] = df_0['n_banheiros'].astype('float')
df_0['n_fotos'] = df_0['n_fotos'].astype('float')
df_0['n_garagens'] = df_0['n_garagens'].astype('float')
df_0['n_quartos'] = df_0['n_quartos'].astype('float')

In [8]:
df_0[df_0.select_dtypes(['category']).columns] = df_0.select_dtypes(['category']).apply(lambda x: x.cat.codes)

In [9]:
df_0 = df_0.drop_duplicates()

In [10]:
from fancyimpute import KNN

df_0_knn = KNN(k=2).fit_transform(df_0)

Imputing row 1/918 with 0 missing, elapsed time: 0.357
Imputing row 101/918 with 0 missing, elapsed time: 0.358
Imputing row 201/918 with 0 missing, elapsed time: 0.360
Imputing row 301/918 with 0 missing, elapsed time: 0.362
Imputing row 401/918 with 0 missing, elapsed time: 0.362
Imputing row 501/918 with 0 missing, elapsed time: 0.363
Imputing row 601/918 with 0 missing, elapsed time: 0.363
Imputing row 701/918 with 0 missing, elapsed time: 0.363
Imputing row 801/918 with 0 missing, elapsed time: 0.364
Imputing row 901/918 with 0 missing, elapsed time: 0.365


In [11]:
df_0 = pd.DataFrame(data=df_0_knn[0:,0:],columns=df_0.columns)
df_0.isna().sum()

find_condominio       0
find_iptu             0
find_loc              0
find_m_quadrado       0
find_price            0
find_vendedor         0
is_academia           0
is_apertamento        0
is_ar_condicionado    0
is_churrasqueira      0
is_elevador           0
is_mobiliado          0
is_piscina            0
is_portaria           0
is_profissional       0
is_salao_festa        0
is_varanda            0
n_banheiros           0
n_fotos               0
n_garagens            0
n_quartos             0
Bairro                0
CEP                   0
dtype: int64

In [12]:
df_0.head(1)

,find_condominio,find_iptu,find_loc,find_m_quadrado,find_price,find_vendedor,is_academia,is_apertamento,is_ar_condicionado,is_churrasqueira,...,is_portaria,is_profissional,is_salao_festa,is_varanda,n_banheiros,n_fotos,n_garagens,n_quartos,Bairro,CEP
0,1100.0,180.0,365.0,105.0,380000.0,26.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,3.0,20.0,0.0,3.0,0.0,305.0


In [13]:
df_1 = df_0.copy()

vamos escolher a coluna de preço para fazer regressão

In [14]:
y = df_1['find_price']

In [15]:
X = df_1.drop(columns=['find_price'])

## Separar os dados em treinamento, validação e teste

temos que revisar as proporções que o professor que que use, eu lembro dele ter dito que queria, k folds

In [16]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

## Selecionar 4 algoritmos de acordo com a tarefa escolhida no passo 1

In [17]:
from sklearn.neural_network import MLPRegressor

In [18]:
from sklearn.ensemble  import RandomForestRegressor
rfr = RandomForestRegressor()

In [19]:
from sklearn.ensemble  import GradientBoostingRegressor
gbr = GradientBoostingRegressor()

In [20]:
from sklearn.neighbors   import KNeighborsRegressor
knnr = KNeighborsRegressor()

## Adicionar MLFlow no treinamento dos modelos para rastreamento

In [21]:
import mlflow
import mlflow.sklearn 

In [22]:
mlflow.set_experiment("nome_do_seu_experimento")

<Experiment: artifact_location='file:///c:/Users/davim/OneDrive/Desktop/playground/scraping/notebook_analysis/mlruns/964034627453830662', creation_time=1720291533006, experiment_id='964034627453830662', last_update_time=1720291533006, lifecycle_stage='active', name='nome_do_seu_experimento', tags={}>

## Executar uma ferramenta de seleção de hiper-parâmetros sobre o conjunto de validação

In [23]:
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical, Real
from skopt.utils import use_named_args
from skopt import gp_minimize

Selecionar poucos hiper-parâmetros por algoritmo (max. 3)

#### RandomForestRegressor

In [25]:
search_space = [
    Categorical(["squared_error", "absolute_error", "friedman_mse", "poisson"], name='criterion'), 
    Categorical(["sqrt", "log2"], name='max_features'), 
]

In [28]:
@use_named_args(search_space)
def objective_rfr(**params):
    rfr = RandomForestRegressor(
        criterion=params['criterion'],
        max_features=params['max_features'],
        random_state=42
    )
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(rfr, X, y, cv=kf, scoring='neg_mean_squared_error')
    return -np.mean(scores)

In [29]:
result = gp_minimize(
    func=objective_rfr,
    dimensions=search_space,
    n_calls=50,
    n_random_starts=10,
    random_state=42
)

# Print the best-found parameters and score
print("Best score: ", result.fun)
print("Best parameters: ", result.x)

c:\Users\davim\OneDrive\Desktop\playground\scraping\myenv\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['poisson', 'log2'] before, using random point ['poisson', 'log2']
  warnings.warn(
c:\Users\davim\OneDrive\Desktop\playground\scraping\myenv\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['poisson', 'sqrt'] before, using random point ['squared_error', 'log2']
  warnings.warn(
c:\Users\davim\OneDrive\Desktop\playground\scraping\myenv\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['poisson', 'log2'] before, using random point ['friedman_mse', 'log2']
  warnings.warn(
c:\Users\davim\OneDrive\Desktop\playground\scraping\myenv\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point ['poisson', 'log2'] before, using random point ['poisson', 'sqrt']
  warnings

Best score:  43341750192.05153
Best parameters:  ['poisson', 'sqrt']


#### mlp

In [ ]:
search_space = [
    Integer(1, 3, name='hidden_layer_sizes'),  # Number of hidden layers
    Categorical(['tanh', 'relu'], name='activation'),  # Neurons in each hidden layer
    Categorical(['constant', 'adaptive'], name='learning_rate_init')  # Initial learning rate
]

In [ ]:
@use_named_args(search_space)
def objective_mlpr(**params):
    mlpr = MLPRegressor(
        hidden_layer_sizes=params['hidden_layer_sizes'],
        activation=params['activation'],
        learning_rate=params['learning_rate_init'],
        max_iter=1000,
        random_state=42
    )
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(mlpr, X, y, cv=kf, scoring='neg_mean_squared_error')
    return -np.mean(scores)

Selecionar o modelo (algoritmo+valores de hiper-parâmetros) com
melhor resultado na métrica de avaliação

In [ ]:
# Perform Bayesian optimization
result = gp_minimize(
    func=objective_mlpr,
    dimensions=search_space,
    n_calls=50,
    n_random_starts=10,
    random_state=42
)

# Print the best-found parameters and score
print("Best score: ", result.fun)
print("Best parameters: ", result.x)

c:\Users\davim\OneDrive\Desktop\playground\scraping\myenv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\davim\OneDrive\Desktop\playground\scraping\myenv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\davim\OneDrive\Desktop\playground\scraping\myenv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\davim\OneDrive\Desktop\playground\scraping\myenv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reac

Executar o melhor modelo de cada algoritmo no conjunto de teste e
selecionar o melhor modelo geral na métrica de avaliação

## Realizar diagnóstico do melhor modelo geral da etapa 5 e melhorá-lo a partir do diagnóstico

Bônus: Utilizar auto-sklearn, clustering para entendimento dos
dados